In [10]:
from torchaudio.models.decoder import ctc_decoder
import numpy as np 
import torch
from torch.nn.functional import softmax, log_softmax
from brainaudio.inference.eval_metrics import _cer_and_wer
from brainaudio.inference.lm_funcs import normalize_shorthand
import pandas as pd

In [11]:
language_model_path = "/data2/brain2text/lm/"
units_txt_file_pytorch = f"{language_model_path}units_pytorch.txt"
imagineville_vocab_phoneme = f"{language_model_path}vocab_lower_100k_pytorch_phoneme.txt"

In [8]:
with open(units_txt_file_pytorch, "r", encoding="utf-8") as f:
    # Use .strip() to remove the newline character from every line
    units = [line.strip() for line in f]
    
units[24]

'NG'

In [12]:
decoder = ctc_decoder(tokens=units_txt_file_pytorch, lexicon=imagineville_vocab_phoneme, 
                       beam_size=300, nbest=20, lm="/data2/brain2text/lm/lm_dec19_huge_4gram.kenlm", 
                       lm_weight=2.0, word_score=0.1, log_add=True, sil_score=0, beam_threshold=1e3)

[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached l

In [13]:
val_transcripts_24 = pd.read_pickle("/data2/brain2text/b2t_24/transcripts_val_cleaned.pkl")
val_transcripts_25 = pd.read_pickle("/data2/brain2text/b2t_25/transcripts_val_cleaned.pkl")

In [8]:
model_logits = np.load("/data2/brain2text/b2t_25/logits/tm_transformer_b2t25_bidirectional/logits_val.npz")

FileNotFoundError: [Errno 2] No such file or directory: '/data2/brain2text/b2t_25/logits/tm_transformer_b2t25_bidirectional/logits_val.npz'

In [ ]:
acoustic_scale = 0.6
ground_truth_arr = []
pred_arr = []
for idx in range(len(model_logits)):
    if idx % 100 == 0:
        print(idx)
    single_trial_logits = torch.as_tensor(model_logits[f'arr_{idx}']).float().unsqueeze(0)
    beam_search_char = decoder(single_trial_logits * acoustic_scale)
    beam_search_transcript_char = normalize_shorthand(" ".join(beam_search_char[0][0].words).strip())
    ground_truth_sentence = val_transcripts[idx]
    ground_truth_arr.append(ground_truth_sentence)
    pred_arr.append(beam_search_transcript_char)
    
cer, wer, wer_sent = _cer_and_wer(pred_arr, ground_truth_arr)
print(wer)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
0.10164107993647432


In [46]:
for p, g in zip(pred_arr, ground_truth_arr):
    
    print(g, " ||||  ", p)

theocracy reconsidered  ||||   they aka recanted
rich purchased several signed lithographs  ||||   take purchase ever said lithographs
so rules we made in unabashed collusion  ||||   so rules we made in invite collusion
lori's costume needed black gloves to be completely elegant  ||||   try some needed back loves to be completely ellen
the tooth fairy forgot to come when roger's tooth fell out  ||||   the truth i forgot to come run washers tooth fall out
that stinging vapor was caused by chloride vaporization  ||||   that singing vapor power was so the kid paper i decision
before thursday's exam review every formula  ||||   before this i am reveal over formal
wildfire near sunshine forces park closures  ||||   ride for near sundown for park
the word means it won't boil away easily nothing else  ||||   the warrant means it won't pull away helly nothing ass
would a blue feather in a man's hat make him happy all day  ||||   would a blue feather in a man's hat make him about all day
he tal

In [13]:
lm_weights = [1.5, 2, 2.5, 3]   
acoustic_score = [0.5, 0.6, 0.7, 0.8, 0.9]
word_penalty = [0, 0.1, -0.1, 0.2, -0.2, 0.3, -0.3]
sil_score = [0, -0.1, -0.2, -0.3]
beam_size = [100]

wer_dict = {}
wer_dict['sil'] = []
wer_dict['lmw'] = []
wer_dict['wp'] = []
wer_dict['bs'] = []
wer_dict['ac'] = []

wer_dict['wer'] = []

for sil in sil_score:
    for wp in word_penalty:
      for lmw in lm_weights:
          for bs in beam_size:
          
            decoder = ctc_decoder(tokens=units_txt_file_pytorch, lexicon=imagineville_vocab_phoneme, 
                       beam_size=bs, nbest=1, lm="/data2/brain2text/lm/lm_dec19_huge_4gram.kenlm", 
                       lm_weight=lmw, word_score=wp, sil_score=sil, log_add=True, beam_threshold=1e3)

            for ac in acoustic_score:
              
              ground_truth_arr = []
              pred_arr = []
          
              for idx in range(880):
                  single_trial_logits = torch.as_tensor(model_logits[f'arr_{idx}']).float().unsqueeze(0)
                  beam_search_outs = decoder(single_trial_logits*ac)
                  beam_search_transcript = normalize_shorthand(" ".join(beam_search_outs[0][0].words).strip())
                  ground_truth_sentence = val_transcripts[idx]
                  ground_truth_arr.append(ground_truth_sentence)
                  pred_arr.append(beam_search_transcript)
                  
              cer, wer, wer_sent = _cer_and_wer(pred_arr, ground_truth_arr)
              
              
              wer_dict['sil'].append(sil)
              wer_dict['lmw'].append(lmw)
              wer_dict['wp'].append(wp)
              wer_dict['bs'].append(bs)
              wer_dict['ac'].append(ac)
              wer_dict['wer'].append(wer)
              
              
              print(f"sil score: {sil}, lm weight: {lmw}, word penalty: {wp}, beam size: {bs}, acoustic score: {ac},  wer: {wer}")


NameError: name 'imagineville_vocab' is not defined